In [3]:
from tagger.viewsets.received_items import ReceivedItemViewSet
from tagger.models.received_item import ReceivedItem, OrderType
from tagger.core.mongo.models.product import Product
from tagger.core.mongo.models.alloff_product import AlloffProduct
from tagger.core.mongo.models.order import Order
from tagger.models import ExtendedOrder
from bson import ObjectId
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [109]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [28]:
with open("../Downloads/거래내역조회_입출식 예금_출력용_20211125.xls", "rb") as f:
    soup = BeautifulSoup(f)
df = pd.read_html(str(soup.find_all("table")[1]), match="거래일시", flavor="html5lib")[0]
df.dropna(subset=["No"], inplace=True)

In [39]:
df[["상대계좌번호", "상대은행"]].drop_duplicates()

,상대계좌번호,상대은행
0,9001192668,케이뱅크
1,561-018621-01-031,기업은행
3,NaN,한국씨티은행
4,085250006851,국민은행
5,3333134908912,카카오뱅크
...,...,...
549,120211054015,국민은행
555,603-158660-97-751,NaN
556,018-914549-93-847,NaN
558,07216788212201,우리은행


In [50]:
df.drop_duplicates(subset=["상대계좌예금주명", "상대계좌번호", "상대은행"]).to_clipboard()

In [46]:
df.loc[df['상대계좌예금주명'] == "김지은"]

,No,거래일시,출금,입금,거래후 잔액,거래내용,상대계좌번호,상대은행,거래구분,수표어음금액,CMS코드,상대계좌예금주명
507,508.0,2021-09-30 17:44:04,40900,0,53837917.0,마담_김지은,3029390482261,농협은행,기업스마트뱅킹,0.0,NaN,김지은


0             9001192668
1      561-018621-01-031
2            85250006851
3          3333010686739
4      250-018435-01-042
             ...        
136         120211054015
137    603-158660-97-751
138    018-914549-93-847
139        7216788212201
140       34650104289401
Name: account_no, Length: 141, dtype: object

,Unnamed: 0,transferred_at,transfer_amount,bank,account_no,user,memo,account_holder,tranfer_type,account_type,account_no_2
0,0,2021-11-25 17:42:11,900,케이뱅크,9001192668,주식회사레스버터,케이뱅크계좌개설,（주）레스버터,기타비용,임대사업자,9001192668
1,1,2021-11-25 17:17:08,109000,기업은행,561-018621-01-031,안장현,소싱_4A8XP,김소영,매입,마담,56101862101031
2,3,2021-11-25 12:02:55,550000,국민은행,85250006851,주식회사 레스버터,김재국님자문비용,김영환（디지탈코리아,임직원,김재국,85250006851
3,4,2021-11-24 16:52:11,50000000,카카오뱅크,3333010686739,주식회사 레스버터,정주영가수금회수1차,정주영,임직원,정주영,3333010686739
4,5,2021-11-24 01:44:24,104700,기업은행,250-018435-01-042,안장현,소싱_4D8FT,주)수인터내셔날,매입,공식몰,25001843501042
...,...,...,...,...,...,...,...,...,...,...,...
136,395,2021-09-18 12:21:55,55275,국민은행,120211054015,주식회사 레스버터,정창해님 카드환불금,정창해,매출,환불,120211054015
137,396,2021-09-07 13:27:04,16750,기업은행,603-158660-97-751,주식회사 레스버터,지방세,서울지방레스버,세금,세금,60315866097751
138,397,2021-09-07 13:26:36,167660,기업은행,018-914549-93-847,주식회사 레스버터,근소세,국세_주식회사 레스버,세금,세금,01891454993847
139,398,2021-09-06 14:04:57,72000,우리은행,7216788212201,주식회사 레스버터,문선영님상품,박선영,매출,환불,7216788212201


In [157]:
df2 = pd.read_excel("21-09-01_21-11-25_transfer_tagged.xlsx")
df2["account_no_2"] = df2["account_no"].apply(lambda x: x.replace("-", "") if type(x) == str else str(x))
df["상대계좌번호_2"] = df["상대계좌번호"].apply(lambda x: x.replace("-", "") if type(x) == str else str(x))
df3 = df.merge(df2, right_on=["account_no_2"], left_on=["상대계좌번호_2"], how="left")

madams = list(df2[df2["account_type"] == "마담"]["account_holder"].drop_duplicates())
team = ["이석민", "김명규", "정주영", "안장현", "전성현", ]
sw = ["(주)한글과컴퓨터", "구글클라우드코리아",]
malls = ["（주）인동에프엔", "(주)한섬", "(주)대현", "주)수인터내셔날", "주식회사시선인터내", "주식회사　바바더닷컴"]
refunds = ["한국사이버결제", "모빌리언스환불", "ＫＣＰ매매보호"]

df3["account_type"] = df3.apply(lambda r: "마담" if r["거래내용"] in madams else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "PG" if r["출금"] == 0 and r["거래내용"] in ["(주)다날", "카카오페이"] else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "카드대금" if "현대카드" in r["거래내용"] and np.isnan(r["account_type"]) else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "카드대금" if "롯데카드" in r["거래내용"] and np.isnan(r["account_type"]) else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "세무사" if r["거래내용"] == "세무법인혜움" else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "임직원" if r["상대계좌예금주명"] in team or r["거래내용"] in team else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "세금" if "국민" in r["거래내용"] else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "SW" if r["거래내용"] in sw else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "물류비용" if r["거래내용"] in ["주)비지에프네트웍스",]else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "투자자" if r["거래내용"] in ["양진철",]else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "공식몰" if r["거래내용"] in malls or r["상대계좌예금주명"] in malls else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "잡비용" if r["거래내용"] in ["공인인증수수료", ] else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "오프라인몰" if r["거래내용"] in ["문선영님상품", ] else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "환불" if "환불" in r["거래내용"] and r["출금"] > 0 else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "잡이익" if "결산" in r["거래내용"] and r["출금"] == 0 else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "자사환불" if r["거래내용"] in refunds and r["출금"] == 0 else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "자문료" if "김재국" in r["거래내용"] else r["account_type"], axis=1)
df3["account_type"] = df3.apply(lambda r: "?" if r["거래내용"] in ["이주연", "김민선"] and r["출금"] == 0 else r["account_type"], axis=1)

In [158]:
df3.to_clipboard()

In [160]:
valid_orders = [x for x in Order.objects(orderstatus__nin=[
    "CREATED", "RECREATED", "PENDING", "PAYMENT_PENDING", "CANCEL_FINISHED",
]).all()]

In [163]:
sum([x.totalprice for x in valid_orders])

150520196